# Import

In [7]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Scaling 클래스
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

#통계 분석 패키지
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

#회귀분석
from statsmodels.formula.api import ols

#qqplot, 회귀분석 상수항 추가
from statsmodels.api import qqplot, add_constant

#선형 모델 formula(y ~ x1+x2+..)
import statsmodels.formula.api as smf

#다중공선성
from statsmodels.stats.outliers_influence import variance_inflation_factor

#학습용/평가용 데이터 분리
from sklearn.model_selection import train_test_split, GridSearchCV

# 선형회귀 모형
from sklearn.linear_model import LinearRegression

#변수선택법, 후진제거법
from sklearn.feature_selection import RFE

#scale변환 -> 표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#평가함수
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

#ridge,lasso,elasticnet 패키지 불러오기
from sklearn.linear_model import Ridge, Lasso, ElasticNet

#export_graphviz : 트리 구조 생성 및 저장
from sklearn.tree import export_graphviz

# 나무 구조 시각화
import graphviz

#예측/회귀
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

#모델 성능 평가
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score,roc_curve,auc,precision_score,recall_score
#confusion matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

#Scale 변환:평균,표준편차 기준
from sklearn.preprocessing import StandardScaler
#주성분 분석
from sklearn.decomposition import PCA
#회귀분석
import statsmodels.api as sm
#RMSE:root mean squared error 함수
def root_mean_squared_error(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))
#MAPE:mean absolute percentage error 함수
def mean_absolute_percentage_error(y_true,y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100
# 폰트
import os 
# 그래프 한글폰트 적용:맑은 고딕
matplotlib.rc("font", family = "Malgun Gothic")
matplotlib.rc("axes", unicode_minus=False)

# Data Load

In [13]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_df.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


# Data Split

In [27]:
train_x = train_df.drop(columns=['LINE', 'PRODUCT_CODE', 'PRODUCT_ID', 'TIMESTAMP','Y_Quality', 'Y_Class'])
train_y = train_df['Y_Class']

# Data Pre-processing

In [28]:
train_x = train_x.fillna(train_x.mean())
train_x = train_x.fillna(0)

# Over-Sampling : SMOTE

In [31]:
# 샘플링 : Over-sampling 등 (필요시 !pip install imblearn로 설치)
# !pip install imblearn
from imblearn.over_sampling import SMOTE

# 목표변수 빈도 확인
print(train_y.value_counts(1),"\n")

# Over-sampling 설정
sm = SMOTE(sampling_strategy='auto')
# train데이터를 이용한 Over-sampling
x_resampled, y_resampled = sm.fit_resample(train_x,train_y)

# 결과 확인
print('----------------------------------------------\n')
print('Over-Sampling 전:\n', train_y.value_counts(),"\n")
print('Over-Sampling 후 Train X: {}'.format(x_resampled.shape))
print('Over-Sampling 후 Train Y: {} \n'.format(y_resampled.shape))
print("Over-Sampling 후 '2':{}".format(sum(y_resampled==2)))
print("Over-Sampling 후 '1':{}".format(sum(y_resampled==1)))
print("Over-Sampling 후 '0':{}".format(sum(y_resampled==0)))

# 데이터 결합
df_resampled = pd.concat([x_resampled,y_resampled], axis=1)

1    0.680602
2    0.172241
0    0.147157
Name: Y_Class, dtype: float64 

----------------------------------------------

Over-Sampling 전:
 1    407
2    103
0     88
Name: Y_Class, dtype: int64 

Over-Sampling 후 Train X: (1221, 2875)
Over-Sampling 후 Train Y: (1221,) 

Over-Sampling 후 '2':407
Over-Sampling 후 '1':407
Over-Sampling 후 '0':407
